In [ ]:
!git clone https://github.com/Q-Future/Q-Align.git
%cd Q-Align
!pip install -e ".[train]"
!pip install flash_attn --no-build-isolation

fatal: destination path 'Q-Align' already exists and is not an empty directory.
/workspace/Q-Align


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Obtaining file:///workspace/Q-Align
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
Checking if build backend supports build_editable ... done
  Building editable for q_align (pyproject.toml) ... done
  Created wheel for q_align: filename=q_align-1.2.0-0.editable-py3-none-any.whl size=7984 sha256=f608ed61b463e639653627463b7ce00ae11b9c2459311f03185e145ff1c581c4
  Stored in directory: /tmp/pip-ephem-wheel-cache-s9h64smn/wheels/6f/99/0b/dd230991014892200c16f9bfed662d47ee4b125637a86ed7ae
Successfully built q_align
  Attempting uninstall: q_align
    Found existing installation: q_align 1.2.0
    Uninstalling q_align-1.2.0:
      Successfully uninstalled q_align-1.2.0

[notice] A new release of pip is available: 23.3.1 -> 24.1.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 

In [ ]:
!pip install deepspeed


[notice] A new release of pip is available: 23.3.1 -> 24.1.2
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
# Copy JSON file
!cp /workspace/train_split_1.json /workspace/Q-Align/playground/data/ft/

# Create directory for video data
!mkdir -p /workspace/video_data

# Copy video files
!cp /workspace/*.mp4 /workspace/video_data/
# If you have GIF files, use this command as well:
# !cp /content/drive/MyDrive/Q-Align_data/*.gif /content/video_data/

In [ ]:
!ls -l /workspace/Q-Align/playground/data/ft/
!ls -l /workspace/video_data/

total 36
drwxr-xr-x 2 root root 4096 Jul 16 15:54 agi
drwxr-xr-x 2 root root 4096 Jul 16 15:54 agiqa3k
drwxr-xr-x 2 root root 4096 Jul 16 15:54 cgi
drwxr-xr-x 2 root root 4096 Jul 16 15:54 csiq
drwxr-xr-x 2 root root 4096 Jul 16 15:54 live
drwxr-xr-x 2 root root 4096 Jul 16 15:54 livec
drwxr-xr-x 2 root root   69 Jul 16 15:54 maxwell
-rw-r--r-- 1 root root 8887 Jul 18 00:12 train_split_1.json
total 32828
-rw-r--r-- 1 root root 1367318 Jul 18 00:12  Anatomy_Bad.mp4
-rw-r--r-- 1 root root 1730556 Jul 18 00:12  Beach_Good.mp4
-rw-r--r-- 1 root root  710597 Jul 18 00:12  BeachwithProduct_Good.mp4
-rw-r--r-- 1 root root 3021685 Jul 18 00:12  Car_Moving_Backlwards.mp4
-rw-r--r-- 1 root root  826721 Jul 18 00:12  ExtraHand_Bad.mp4
-rw-r--r-- 1 root root  995400 Jul 18 00:12  EyesDistorted_Bad.mp4
-rw-r--r-- 1 root root  238587 Jul 18 00:12  FLowers_anatomy.mp4
-rw-r--r-- 1 root root 1077436 Jul 18 00:12  Face_Bad.mp4
-rw-r--r-- 1 root root 3294863 Jul 18 00:12  Girl_Good.mp4
-rw-r--r-- 1 root

In [ ]:
%%writefile scripts/my_videos_lora.sh
#!/bin/bash

# Create input file with choice
echo "2" > input.txt
export WANDB_API_KEY="d748b17184ce7ec9c05419bb670fbda0fd136a13"

wandb login $WANDB_API_KEY

# Run the script with input redirection
deepspeed q_align/train/train_mem.py < input.txt \
    --deepspeed ./scripts/zero3.json \
    --model_name_or_path q-future/one-align \
    --version v1 \
    --data_path /workspace/Q-Align/playground/data/ft/train_split_1.json \
    --image_folder /workspace/video_data \
    --bf16 True \
    --output_dir /workspace/Q-Align/checkpoints/ \
    --num_train_epochs 1 \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_step 4 \
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 50000 \
    --save_total_limit 1 \
    --learning_rate 2e-5 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --tf32 True \
    --model_max_length 2048 \
    --gradient_checkpointing True \
    --dataloader_num_workers 4 \
    --lazy_preprocess True \
    --report_to wandb


Overwriting scripts/my_videos_lora.sh


In [ ]:
!chmod +x scripts/my_videos_lora.sh
!bash scripts/my_videos_lora.sh

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
[2024-07-18 00:20:31,397] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-07-18 00:20:33,201] [WARNING] [runner.py:196:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2024-07-18 00:20:33,201] [INFO] [runner.py:555:main] cmd = /usr/bin/python -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMCwgMV19 --master_addr=127.0.0.1 --master_port=29500 --enable_each_rank_log=None q_align/train/train_mem.py --deepspeed ./scripts/zero3.json --model_name_or_path q-future/one-align --version v1 --data_path /workspace/Q-Align/playground/data/ft/train_split_1.json --image_folder /workspace/video_data --bf16 True --output_dir /workspace/Q-Align/checkpoints/ --num_train_epochs 1 --per_device_train_batch_size 1 --per_device_eval_batch_size 1 --gradient_accumulation_step 4 --evaluation_strategy no --save_strategy steps --s

In [ ]:
!ls /workspace/Q-Align/checkpoints/

config.json			  q-align-my-videos-lora
pytorch_model-00001-of-00004.bin  special_tokens_map.json
pytorch_model-00002-of-00004.bin  tokenizer.model
pytorch_model-00003-of-00004.bin  tokenizer_config.json
pytorch_model-00004-of-00004.bin  trainer_state.json
pytorch_model.bin.index.json	  training_args.bin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
!pwd


/workspace/Q-Align


In [ ]:
import sys
sys.path.append('/workspace/Q-Align')
from q_align.evaluate.vqa_eval import main
from types import SimpleNamespace

args = SimpleNamespace(
    model_path="/workspace/Q-Align/checkpoints",
    model_base=None,  # Set this to None if loading a full model
    device="cuda:0",
    conv_mode=None,
    temperature=0.2,
    max_new_tokens=512,
    load_8bit=False,
    load_4bit=False,
    debug=False,
    image_aspect_ratio='pad'
)

main(args)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
import os
print(os.listdir('/workspace/Q-Align/checkpoints'))


['q-align-my-videos-lora', 'trainer_state.json', 'config.json', 'pytorch_model-00001-of-00004.bin', 'pytorch_model-00002-of-00004.bin', 'pytorch_model-00003-of-00004.bin', 'pytorch_model-00004-of-00004.bin', 'pytorch_model.bin.index.json', 'tokenizer_config.json', 'special_tokens_map.json', 'tokenizer.model', 'training_args.bin', 'preprocessor_config.json']
